In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import client_surveillance_jupyter

In [4]:
COMPANY_IDENTIFIER = ['RG']
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [5]:
#fetch download report and lisences
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pd.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pd.read_sql_query(company_download_summaries_query, engine)

In [6]:
company_licenses_dataframe[['us_state', 'license_number', 'license_category', 'legal_name']]

,us_state,license_number,license_category,legal_name
0,CA,C11-0000496-LIC,Distributor,"CENTRAL COAST AG DISTRIBUTION, LLC"
1,CA,C11-0001495-LIC,Distributor,"Central Coast Ag Distribution, LLC"
2,CA,C11-0001094-LIC,Distributor,"Central Coast Ag Distribution, LLC"
3,CA,CCL18-0002256,Manufacturer,"Central Coast Ag Farming, LLC"
4,CA,CDPH-10003156,Manufacturer,"Central Coast Ag Products, LLC"
5,CA,CDPH-10004620,Manufacturer,"Central Coast Ag Products, LLC"
6,CA,C11-0001512-LIC,Distributor,"Central Coast Ag Farming, LLC"
7,CA,C13-0000057-LIC,Distributor,"CENTRAL COAST AG FARMING, LLC"


## Check download status summaries

In [7]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

In [8]:
license_numbers

array(['CCL18-0002256', 'CDPH-10003156', 'CDPH-10004620',
       'C11-0000496-LIC', 'C11-0001094-LIC', 'C11-0001495-LIC',
       'C11-0001512-LIC', 'C13-0000057-LIC'], dtype=object)

In [ ]:
#bad_download_history = client_surveillance_jupyter.check_company_license_download(license_numbers,download_summary_records)

In [ ]:
#bad_download_history

## Choose license numbers

In [9]:
license_numbers = list(license_numbers)

In [10]:
license_numbers

['CCL18-0002256',
 'CDPH-10003156',
 'CDPH-10004620',
 'C11-0000496-LIC',
 'C11-0001094-LIC',
 'C11-0001495-LIC',
 'C11-0001512-LIC',
 'C13-0000057-LIC']

## Pull data

In [11]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_unknown_transfer_packages_query = create_queries.create_company_unknown_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
#     license_numbers=license_numbers,
)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_sales_receipts_query = create_queries.create_company_sales_receipts_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_sales_receipts_with_transactions_query = create_queries.create_company_sales_receipts_with_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
)



company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pd.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_unknown_transfer_packages_dataframe = pd.read_sql_query(company_unknown_transfer_packages_query, engine)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [ ]:
# all of nabis's licenses: ['C11-0001274-LIC','C11-0000150-LIC','C11-0001651-LIC','C11-0000349-LIC','C10-0000085-LIC','CCL18-0000558',\
#'C11-0000340-LIC','CDPH-10003426']

In [13]:
company_outgoing_transfer_packages_dataframe['license_number'].unique()

array(['CCL18-0002256', 'C11-0000496-LIC', 'CDPH-10003156',
       'C11-0001512-LIC', 'C11-0001094-LIC'], dtype=object)

In [14]:
df_out = company_outgoing_transfer_packages_dataframe

In [15]:
df_out['created_date'] = pd.to_datetime(df_out['created_date'])

In [16]:
df_out_nabis = df_out[(df_out['recipient_facility_license_number'] == 'C11-0000825-LIC')]

In [ ]:
l_list= pd.read_csv('recipient_list.csv')

In [ ]:
l_list['name'] = l_list['name'].str.lower()

In [ ]:
l_list.sort_values(by = ['name','type'])

In [ ]:
df_out.head()

In [ ]:
df_out_merge = df_out.merge(l_list,left_on = 'recipient_facility_license_number',right_on = 'license')

In [ ]:
valid_license = list(l_list[(~l_list['type'].isin(['Laboratory']))&(~l_list['name'].str.contains('central'))]['license'].unique())

In [ ]:
valid_license

In [ ]:
lic_0496 = df_out_merge[df_out_merge['shipper_facility_license_number'].isin(['C11-0000496-LIC'])]
lic_1094 = df_out_merge[df_out_merge['shipper_facility_license_number'].isin(['C11-0001094-LIC'])]
lic_1512 = df_out_merge[df_out_merge['shipper_facility_license_number'].isin(['C11-0001512-LIC'])]
cdph_3156 = df_out_merge[df_out_merge['shipper_facility_license_number'].isin(['CDPH-10003156'])]
ccl_2256 = df_out_merge[df_out_merge['shipper_facility_license_number'].isin(['CCL18-0002256'])]

In [ ]:
lic_0496_no_lab_no_self = lic_0496[lic_0496['recipient_facility_license_number'].isin(valid_license)]
cdph_3156_no_lab_no_self = cdph_3156[cdph_3156['recipient_facility_license_number'].isin(valid_license)]
lic_1094_no_lab_no_self = lic_1094[lic_1094['recipient_facility_license_number'].isin(valid_license)]
lic_1512_no_lab_no_self = lic_1512[lic_1512['recipient_facility_license_number'].isin(valid_license)]
ccl_2256_no_lab_no_self = ccl_2256[ccl_2256['recipient_facility_license_number'].isin(valid_license)]

In [ ]:
lic_0496_sum = lic_0496_no_lab_no_self[['name','created_date','package_id']].groupby(['created_date','name']).count().reset_index()

lic_1094_sum = lic_1094_no_lab_no_self[['name','created_date','package_id']].groupby(['created_date','name']).count().reset_index()

lic_1512_sum = lic_1512_no_lab_no_self[['name','created_date','package_id']].groupby(['created_date','name']).count().reset_index()

cdph_3156_sum = cdph_3156_no_lab_no_self[['name','created_date','package_id']].groupby(['created_date','name']).count().reset_index()

ccl_2256_sum = ccl_2256_no_lab_no_self[['name','created_date','package_id']].groupby(['created_date','name']).count().reset_index()




In [ ]:
import seaborn as sns; sns.set()
sns.set(rc={'figure.figsize':(10,5)})

In [ ]:
sns.lineplot(data = ccl_2256_sum, x = 'created_date',y = 'package_id',hue = 'name',\
             marker='o',markersize=8)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.lineplot(data = lic_0496_sum, x = 'created_date',y = 'package_id',hue = 'name',\
             marker='o',markersize=8)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.lineplot(data = lic_1094_sum, x = 'created_date',y = 'package_id',hue = 'name',\
            marker='o',markersize=8)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.lineplot(data = lic_1512_sum, x = 'created_date',y = 'package_id',hue = 'name',\
            marker='o',markersize=8)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.lineplot(data = cdph_3156_sum, x = 'created_date',y = 'package_id',hue = 'name',\
            marker='o',markersize=8)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Inventory

In [ ]:
df_inventory = company_inventory_packages_dataframe

In [ ]:
today = date.today()
today

In [ ]:
COMPANY_IDENTIFIER

In [ ]:
# there are in total 1807 packages in its inventory 6-13-22
df_inventory['license_number'].unique()

In [ ]:
df_inventory['quantity'] = df_inventory['quantity'].round(2)

### inventory quantity

In [ ]:
inv_094 = df_inventory[df_inventory['license_number'] == 'C11-0001094-LIC'][['product_category_name','unit_of_measure','quantity']].groupby(['product_category_name','unit_of_measure']).sum().reset_index()

In [ ]:
inv_094

In [ ]:
# inventory quantity by product category name
inv = df_inventory[['license_number','product_category_name','unit_of_measure','quantity']].groupby(['license_number','product_category_name','unit_of_measure']).sum().sort_values(by = ['product_category_name','unit_of_measure','quantity']).reset_index()




In [ ]:
inv

### inventory freshness

In [ ]:
df_inventory["age"] = [today - df_inventory["packaged_date"][i] for i in range(len(df_inventory))]
df_inventory["age_int"] = [df_inventory["age"][i] / numpy.timedelta64(1, "D")
        for i in range(len(df_inventory))]

In [ ]:
# histogram
ax = plt.hist(df_inventory[df_inventory['product_category_name'] == 'Vape Cartridge (weight - each)']['age_int'],bins = 50)
plt.ylabel('count package_id')
plt.xlabel('age')
plt.title('Vape Cartridge (weight - each)')



### inventory freshness by product categories

In [ ]:
df_inventory[['product_category_name','unit_of_measure','age_int']].groupby(['product_category_name','unit_of_measure']).describe()

In [ ]:
print(company_inventory_packages_dataframe[company_inventory_packages_dataframe['age_int'] <= 90]['package_id'].nunique() / company_inventory_packages_dataframe['package_id'].nunique())
print(company_inventory_packages_dataframe[company_inventory_packages_dataframe['age_int'] <= 180]['package_id'].nunique() / company_inventory_packages_dataframe['package_id'].nunique())
print(company_inventory_packages_dataframe[company_inventory_packages_dataframe['age_int'] <= 270]['package_id'].nunique() / company_inventory_packages_dataframe['package_id'].nunique())
print(company_inventory_packages_dataframe[company_inventory_packages_dataframe['age_int'] <= 360]['package_id'].nunique() / company_inventory_packages_dataframe['package_id'].nunique())



# Turnover

In [ ]:
company_outgoing_transfer_packages_dataframe['created_date'] = pd.to_datetime(company_outgoing_transfer_packages_dataframe['created_date'])

In [ ]:
# recent 3 month outgoing packages 
recent_outgoing_3m = company_outgoing_transfer_packages_dataframe[company_outgoing_transfer_packages_dataframe['created_date'] >= '2022-03-13']
recent_outgoing_1m = company_outgoing_transfer_packages_dataframe[company_outgoing_transfer_packages_dataframe['created_date'] >= '2022-05-13']



In [ ]:
recent_outgoing_3m.head()

In [ ]:
recent_outgoing_3m_package = recent_outgoing_3m[['package_id','product_category_name','shipped_unit_of_measure','shipped_quantity']].\
groupby(['package_id','shipped_unit_of_measure','product_category_name']).sum().reset_index()

recent_outgoing_1m_package = recent_outgoing_1m[['package_id','shipped_unit_of_measure','product_category_name','shipped_quantity']].\
groupby(['package_id','shipped_unit_of_measure','product_category_name']).sum().reset_index()

In [ ]:
recent_outgoing_3m_package.columns = ['package_id','unit_of_measure','product_category_name','shipped_quantity']
recent_outgoing_1m_package.columns = ['package_id','unit_of_measure','product_category_name','shipped_quantity']

In [ ]:
recent_outgoing_1m_package

In [ ]:
recent_outgoing_3m_package[['product_category_name','shipped_quantity','unit_of_measure']].\
groupby(['product_category_name','unit_of_measure']).sum()


In [ ]:
#recent_outgoing_3m_package.to_csv('rg_recent_outgoing_3m_package_0613.csv')

In [ ]:
#recent_outgoing_1m_package.to_csv('rg_recent_outgoing_1m_package_0613.csv')

In [ ]:
joined_3m = company_inventory_packages_dataframe.merge(recent_outgoing_3m,how = 'left', left_on = ['package_id','unit_of_measure'],right_on = ['package_id','shipped_unit_of_measure'])
joined_1m = company_inventory_packages_dataframe.merge(recent_outgoing_1m,how = 'left', left_on = ['package_id','unit_of_measure'],right_on = ['package_id','shipped_unit_of_measure'])




In [ ]:
joined_1m[joined_1m['shipped_quantity'].notnull()]['package_id'].nunique()


In [ ]:
joined_3m[joined_3m['shipped_quantity'].notnull()]['package_id'].nunique()


In [ ]:
move_3m= joined_3m[joined_3m['shipped_quantity'].notnull()]
move_3m['package_id'].nunique()

In [ ]:
move_1m= joined_1m[joined_1m['shipped_quantity'].notnull()]
move_1m['package_id'].nunique()

In [ ]:
move_3m['package_id'].nunique() / company_inventory_packages_dataframe['package_id'].nunique()

In [ ]:
move_1m['package_id'].nunique() / company_inventory_packages_dataframe['package_id'].nunique()

In [ ]:
move_3m['packaged_date'] = pd.to_datetime(move_3m['packaged_date'])
move_3m['diff'] = move_3m['created_date'] - move_3m['packaged_date']
move_3m = move_3m.reset_index()
move_3m["diff_int"] = [move_3m["diff"][i] / numpy.timedelta64(1, "D")
        for i in range(len(move_3m))]

move_1m['packaged_date'] = pd.to_datetime(move_1m['packaged_date'])
move_1m['diff'] = move_1m['created_date'] - move_1m['packaged_date']
move_1m = move_1m.reset_index()
move_1m["diff_int"] = [move_1m["diff"][i] / numpy.timedelta64(1, "D")
        for i in range(len(move_1m))]

In [ ]:
move_3m[['product_category_name_x','diff_int','unit_of_measure']].\
groupby(['product_category_name_x','unit_of_measure']).describe()

In [ ]:
move_1m[['product_category_name_x','diff_int','unit_of_measure']].\
groupby(['product_category_name_x','unit_of_measure']).describe()

# HERBL

In [ ]:
COMPANY_IDENTIFIER = ['HBL']
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [ ]:
#fetch download report and lisences
company_licenses_query = create_queries.create_company_licenses_query(COMPANY_IDENTIFIER)
company_download_summaries_query = create_queries.create_company_download_summaries_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_licenses_dataframe = pd.read_sql_query(company_licenses_query, engine)
company_download_summaries_dataframe = pd.read_sql_query(company_download_summaries_query, engine)

In [ ]:
company_licenses_dataframe

In [ ]:
license_numbers = company_download_summaries_dataframe['license_number'].unique()
download_summary_records = company_download_summaries_dataframe.to_dict('records')

In [ ]:
license_numbers = list(license_numbers)
license_numbers

In [ ]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    license_numbers=license_numbers,
)
company_unknown_transfer_packages_query = create_queries.create_company_unknown_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
#     license_numbers=license_numbers,
)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_sales_receipts_query = create_queries.create_company_sales_receipts_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_sales_receipts_with_transactions_query = create_queries.create_company_sales_receipts_with_transactions_query(
    COMPANY_IDENTIFIER,
    SALES_TRANSACTIONS_START_DATE,
    license_numbers=license_numbers,
)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    license_numbers=license_numbers,
)



company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pd.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_unknown_transfer_packages_dataframe = pd.read_sql_query(company_unknown_transfer_packages_query, engine)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [ ]:
company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['shipper_facility_license_number'].isin([]

In [ ]:
h_set = set(company_incoming_transfer_packages_dataframe['shipper_facility_license_number'])

In [ ]:
company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['shipper_facility_license_number'] == 'C11-0000496-LIC']




In [ ]:
company_incoming_transfer_packages_dataframe['product_name_lower'] = company_incoming_transfer_packages_dataframe['product_name'].str.lower()

In [ ]:
company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['product_name_lower'].str.contains('raw garden')]['shipper_facility_license_number'].unique()

In [ ]:
company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['shipper_facility_license_number'] == 'C11-0001135-LIC']['product_name'].unique()

In [ ]:
r_set = set(['CDPH-10003156', 'C8-0000013-LIC', 'C11-0000825-LIC',
       'C11-0000050-LIC', 'C11-0000496-LIC', 'CDPH-10003334',
       'C11-0001094-LIC', 'C11-0000777-LIC', 'C8-0000012-LIC',
       'C9-0000402-LIC', 'C11-0001309-LIC', 'C11-0000955-LIC',
       'C8-0000040-LIC', 'CDPH-10003309', 'C11-0001135-LIC',
       'CCL18-0002256', 'C11-0001269-LIC', 'CDPH-10004061',
       'CDPH-10003223'])

In [ ]:
r_set.intersection(h_set)

In [ ]:


company_incoming_transfer_packages_dataframe[company_incoming_transfer_packages_dataframe['package_id'] == 25903040]